In [1]:
import os
import pandas as pd
import folium
from geopy.geocoders import Photon, Nominatim, GoogleV3
from geopy.exc import GeocoderTimedOut, GeocoderUnavailable

import urllib.parse

from dotenv import load_dotenv

In [2]:
# Load environment variables from .env file
load_dotenv()

# Read the API key from the environment variable. It is important to keep this safe.
google_api_key = os.getenv('GOOGLE_API_KEY')

In [3]:
df = pd.read_csv("./buildings_annarbor.csv")# read the file generate in the first notebook

In [4]:
# Combine address components into a single address string
df['Address'] = df['Number'].astype(str) + ' ' + df['Street Name'] + ', ' + df['city'] + ', ' + df['state']

In [5]:
# df = df[df.Index == 42] #if you want to test first for a small part of the dataset
# df

In [6]:

# Initialize geolocator
# For photon and nominatim some address are missin (numbers), and lead for wrong results. 
# Maybe adding the zipcode and testing more it will be possible to work around this.

geolocator_google = GoogleV3(api_key=google_api_key)
geolocator_photon = Photon(timeout=10)
geolocator_nominatim = Nominatim(user_agent="my_app")

In [7]:
# Function to geocode an address
def geocode_address(address):
    try:
        # Attempt geocoding with Google first        
        location = geolocator_google.geocode(address)
        if location:
            # Extract the zip code and country
            zipcode, country = None, None
            for component in location.raw['address_components']:
                if 'postal_code' in component['types']:
                    zipcode = component['long_name']
                if 'country' in component['types']:
                    country = component['long_name']

            # Construct the Google Maps link using coordinates
            link_coords = f"https://maps.google.com/?q={location.latitude},{location.longitude}"

            # Construct the Google Maps link using the address
            address_encoded = urllib.parse.quote(address)
            link_address = f"https://www.google.com/maps?q={address_encoded}"
            return [location.latitude, location.longitude, zipcode, country, link_address, link_coords]
        
        # Attempt geocoding with Photon
        location = geolocator_photon.geocode(address)
        if location:
            # Extract the zip code and country if available
            zipcode = location.raw['properties'].get('postcode', None)
            country = location.raw['properties'].get('countrycode', None).upper()

            # Construct the Bing Maps link using coordinates
            link_coords = f"https://www.bing.com/maps?q={location.latitude},{location.longitude}"

            # Construct the Bing Maps link using the address
            address_encoded = urllib.parse.quote(address)
            link_address = f"https://www.bing.com/maps?q={address_encoded}"
            return [location.latitude, location.longitude, zipcode, country, link_address, link_coords]
        
        # Attempt geocoding with Nominatim
        location = geolocator_nominatim.geocode(address)
        if location:
            # Extract the zip code and country if available
            zipcode = location.raw['address'].get('postcode', None)
            country = location.raw['address'].get('country', None)

            # Construct the Bing Maps link using coordinates
            link_coords = f"https://www.bing.com/maps?q={location.latitude},{location.longitude}"

            # Construct the Bing Maps link using the address
            address_encoded = urllib.parse.quote(address)
            link_address = f"https://www.bing.com/maps?q={address_encoded}"
            return [location.latitude, location.longitude, zipcode, country, link_address, link_coords]
        
        # If all geocoding services fail, return None or handle as needed
        return None
    
    except (GeocoderTimedOut, GeocoderUnavailable) as e:
        # Handle specific geocoding errors
        print(f"Geocoding error for address {address}: {e}")
        return None
    except Exception as e:
        # Handle any unexpected errors
        print(f"Unexpected error for address {address}: {e}")
        return None



In [8]:
# df = df.head(1).copy()

In [9]:
# Apply the function to each address and expand the results into separate columns
df[['Latitude', 'Longitude', 'Zipcode', 'Country', 'Link_Address', 'Link_Coordinates']] = df['Address'].apply(geocode_address).apply(pd.Series)

In [10]:
df = df.reset_index() #making sure that the index is starating at zero

In [11]:
# Initialize map, preview the results. Double check if there are any errors in location
m = folium.Map(location=[df['Latitude'][0], df['Longitude'][0]], zoom_start=15)

# Add markers to the map
for i, row in df.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']],
                  popup=row['Address']).add_to(m)

# Save the map to an HTML file
# m.save('map.html')

# Display the map
m

In [12]:
df.to_csv('buildings_annarbor_coordinates.csv', index=False) #save the results as csv